<a href="https://colab.research.google.com/github/Idk507/Qwen_2_VL/blob/main/ChatWithVideo_Qwen2VL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers accelerate flash_attn
!pip install qwen_vl_utils av

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-h81elsvi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-h81elsvi
  Resolved https://github.com/huggingface/transformers to commit e782e95e3465b66a377c0a7fe95f8f10cd87459d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 69.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9964918 sha256=51774098c03edfb070008ee3663cce7b19f8def33fce75d9d439b05d1bc4d743
  Stored in directory: /tmp/pip-ephem-wheel-cache-h2orycp2/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
  Created wheel for flash_attn: filename=flash_attn-2.

In [2]:
from transformers import Qwen2VLForConditionalGeneration,AutoTokenizer,AutoProcessor
from qwen_vl_utils import process_vision_info
import torch



In [3]:
model_name = "Qwen/Qwen2-VL-2B-Instruct"



In [4]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,torch_dtype= torch.bfloat16,device_map="auto",attn_implementation="flash_attention_2"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

In [5]:
processor = AutoProcessor.from_pretrained(model_name)

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [6]:
message =[
    {
        "role":"user",
        "content" :[
            {
                "type":"video",
                "video" : "",
                "max_pixels": 360*420,
                "fps":1.0
            },
            {
                "type" :"text","text":"Why did the doctor mark on the top of knee area?"
            }
        ]
    }
]

In [ ]:
text = processor.apply_chat_template(message,add_generation_prompt=True)


In [ ]:
image_inputs,video_inputs = process_vision_info(message)

inputs = processor(
    text = [text],
    images = image_inputs,
    videos = video_inputs,
    return_tensors = "pt",
    padding=True
)


In [ ]:
inputs = inputs.to("cuda")

In [ ]:
generated_ids = model.generate(
    **inputs,
    max_new_tokens = 512)


In [ ]:
generated_ids_trimmed = [
    x[len(in_ids):] for in_ids,x in zip(inputs["input_ids"],generated_ids)
]

In [ ]:
output_text = processor.batch_decode(generated_ids_trimmed,skip_special_tokens=True)
print(output_text)